# コラム: ジオコーディング

In [ ]:
import json
from io import StringIO
from urllib import parse, request

import folium
import geopandas as gpd

place = "区役所"
url = parse.urlunparse(
    (
        "https",
        "msearch.gsi.go.jp",
        "/address-search/AddressSearch",
        None,
        parse.urlencode({"q": place}),
        None,
    )
)
res = request.urlopen(url)
geojson = StringIO(
    json.dumps(
        {
            "type": "FeatureCollection",
            "features": json.loads(res.read().decode()),
        }
    )
)
gdf = gpd.read_file(geojson)
center = (
    gdf.loc[:, "geometry"].y.mean(),
    gdf.loc[:, "geometry"].x.mean(),
)
m = folium.Map(location=center, zoom_start=7)
gdf.apply(
    lambda s: folium.Marker(
        location=[
            s["geometry"].y,
            s["geometry"].x,
        ]
    ).add_to(m),
    axis=1,
)
m